In [ ]:
"""

Apply the uncertainty of the parameters from Burgard et al. 2022 to show that 
uncertainty from parameters is lower than uncertainty from parameterisations

"""


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import pandas as pd 
from tqdm.notebook import trange, tqdm
import time
import multimelt.melt_functions as meltf
import multimelt.plume_functions as pf
import summer_paper.useful_functions as uf
from multimelt.constants import *
import seaborn as sns
import os, glob
import distributed

In [ ]:
#home_path = '/Users/claraburgard/bettik_clara/'
home_path='/bettik/burgardc/'
#home_path='/data/users/burgardc/bettik_clara/'

In [ ]:
mod = 'CNRM-CM6-1'
scenario = 'ssp585'

In [ ]:
to2300 = False

if scenario == 'historical':
    yystart = 1850
    yyend = 2014
else:
    if to2300:
        yystart = 2015
        yyend = 2300
    else:
        yystart = 2015
        yyend = 2100   

READ IN DATA

In [ ]:
inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
inputpath_profiles = home_path+'/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
inputpath_plumes = home_path+'/DATA/SUMMER_PAPER/interim/PLUMES/BedMachine_4km/'
inputpath_boxes = home_path+'/DATA/SUMMER_PAPER/interim/BOXES/BedMachine_4km/'

outputpath_simple_all_old = home_path+'/DATA/BASAL_MELT_PARAM/interim/SIMPLE/nemo_5km_06161821_oneFRIS/'
outputpath_simple_all_new = home_path+'/DATA/SUMMER_PAPER/interim/SIMPLE/nemo_5km_orig_christoph_oneFRIS/'
outputpath_melt = home_path+'/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'


In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachine_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)
file_isf['isf_name'] = file_isf['isf_name'].astype(str)
file_isf['region'] = file_isf['region'].astype(str)

In [ ]:
BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v3_aggregated4km_allvars.nc')
BedMachine_orig_cut = uf.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
file_bed_goodGL = BedMachine_orig_cut['bed']
file_draft = (BedMachine_orig_cut['thickness'] - BedMachine_orig_cut['surface']).where(file_isf['ISF_mask'] > 1)
file_isf_conc = BedMachine_orig_cut['isf_conc']

In [ ]:
file_TS_list = []
for tt in tqdm(range(yystart,yyend+1)):
    file_T_orig = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    file_S_orig = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(tt)+'.nc')
    file_TS_orig = xr.merge([file_T_orig.rename({'thetao':'theta_ocean'}), file_S_orig.rename({'so':'salinity_ocean'})]).sel(Nisf=rignot_isf).assign_coords({'time': tt})
    file_TS_list.append(file_TS_orig)
file_TS = xr.concat(file_TS_list, dim='time').rename({'z':'depth'})
file_TS['depth'] = -1*file_TS['depth']

In [ ]:
lon = file_isf.longitude
lat = file_isf.latitude

In [ ]:
xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area

In [ ]:
ice_draft_pos = file_draft
ice_draft_neg = -ice_draft_pos

In [ ]:
isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')

In [ ]:
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_2D_oneFRIS.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'BedMachine_4km_boxes_1D_oneFRIS.nc')
plume_charac = xr.open_dataset(inputpath_plumes+'BedMachine_4km_plume_characteristics.nc')

In [ ]:
param_var_of_int_2D = file_isf[['ISF_mask', 'latitude', 'longitude', 'dGL']]
param_var_of_int_1D = file_isf[['front_bot_depth_avg', 'front_bot_depth_max','isf_name']]

In [ ]:
geometry_info_2D = plume_charac.merge(param_var_of_int_2D).merge(ice_draft_pos.rename('ice_draft_pos')).merge(grid_cell_area_weighted.rename('grid_cell_area_weighted')).merge(file_isf_conc.rename('isfdraft_conc'))
geometry_info_1D = param_var_of_int_1D

SIMPLE

In [ ]:
gammas_new_tuning = xr.open_dataset(outputpath_simple_all_new + 'gammas_simple_ALL.nc')
gammas_old_tuning = xr.open_dataset(outputpath_simple_all_old + 'gammas_simple_ALL.nc')

In [ ]:
simple_param_types = ['linear_local', 'quadratic_local', 'quadratic_local_locslope',
                      'quadratic_mixed_mean', 'quadratic_mixed_locslope'] 

nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for tuning_sort in ['new']: #'old'
    
    for mparam in simple_param_types:
        
        if tuning_sort == 'old':
            gamma = gammas_old_tuning['slope'].sel(param=mparam)
        elif tuning_sort == 'new':
            gamma = gammas_new_tuning['slope'].sel(param=mparam)

        eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                                geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                                mparam, gamma, U_param=True, 
                                                                box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=True)

        eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+tuning_sort+'tuning_oneFRIS.nc')


In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for tuning_sort in ['new']: #'old',
    
    mparam = 'lazero19'
    
    if tuning_sort == 'old':
        gamma = 2.8e-4
        E0 = 4.2e-2
    elif tuning_sort == 'new':
        gamma = 6.2e-4 
        E0 = 2.0e-2 
        
    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                         geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                         mparam, gamma, E0=E0, 
                                                         box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, verbose=True)
    
    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+tuning_sort+'tuning_oneFRIS.nc')


In [ ]:
nisf_list = geometry_info_1D.Nisf
T_S_profile = file_TS.ffill(dim='depth')

for tuning_sort in ['new']:  #'old',
    
    mparam = 'boxes_4_pismyes_picopno'
    pism_version = 'yes'
    nD_config = 4
        
    if tuning_sort == 'old':
        gamma = 0.39e-5
        C = 20.5e6
    elif tuning_sort == 'new':
        gamma = 0.87e-5 
        C = 2.8e6
        
    eval_1D = meltf.calculate_melt_rate_Gt_and_box1_all_isf(nisf_list, T_S_profile, 
                                                             geometry_info_2D, geometry_info_1D, isf_stack_mask, 
                                                             mparam, gamma, C=C, angle_option='local',
                                                             box_charac_2D=box_charac_all_2D, box_charac_1D=box_charac_all_1D, box_tot=nD_config, 
                                                             box_tot_option='nD_config', pism_version=pism_version, picop_opt='no',
                                                             verbose=True)  
    
    eval_1D.to_netcdf(outputpath_melt+'eval_metrics_1D_'+mparam+'_'+tuning_sort+'tuning_oneFRIS.nc')
